# Data_download and Preparation

This notebook is dedicated to the end-to-end preparation of analysis-ready datasets required to investigate the impact of renewable energy generation on electricity prices in Queensland (QLD) within the National Electricity Market (NEM). It performs the following key tasks:

##### 1. Folder Structure Setup

In [4]:
# Setup - Create directories
import os

base_dir = "data"
raw_dir = os.path.join(base_dir, "raw")
cleaned_dir = os.path.join(base_dir, "cleaned")
reg_dir = os.path.join(base_dir, "registration")

os.makedirs(raw_dir, exist_ok=True)
os.makedirs(cleaned_dir, exist_ok=True)
os.makedirs(reg_dir, exist_ok=True)

print("Folders created:")
print(f"- Raw: {raw_dir}")
print(f"- Cleaned: {cleaned_dir}")
print(f"- Registration: {reg_dir}")

Folders created:
- Raw: data\raw
- Cleaned: data\cleaned
- Registration: data\registration


##### 2. Price Data Download (DISPATCHPRICE)

In [6]:
import time
import requests
import zipfile

# Folder to store zipped & extracted files
mms_dir = os.path.join(raw_dir, "mms_dispatch_price")
os.makedirs(mms_dir, exist_ok=True)

# Setup headers to avoid 403 errors
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to download and extract
def download_dispatchprice(years=(2023, 2024), delay=1):
    base_url = "http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/{}/MMSDM_{}_{:02d}/MMSDM_Historical_Data_SQLLoader/DATA/{}.zip"

    for year in range(years[0], years[1] + 1):
        for month in range(1, 13):
            if year == 2024 and month > 12:
                continue
            month_str = f"{year}{month:02d}"
            filename = f"PUBLIC_DVD_DISPATCHPRICE_{month_str}010000"
            url = base_url.format(year, year, month, filename)

            zip_path = os.path.join(mms_dir, f"{filename}.zip")
            csv_path = os.path.join(mms_dir, f"DISPATCHPRICE_{month_str}.csv")

            # Skip if already downloaded
            if os.path.exists(csv_path):
                print(f"✓ Already exists: {csv_path}")
                continue

            try:
                print(f"⬇ Downloading: {filename}.zip")
                response = requests.get(url, headers=headers)
                response.raise_for_status()

                with open(zip_path, 'wb') as f:
                    f.write(response.content)

                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(mms_dir)

                extracted_csv = os.path.join(mms_dir, f"{filename}.CSV")
                os.rename(extracted_csv, csv_path)
                print(f"Extracted: {csv_path}")

                time.sleep(delay)

            except Exception as e:
                print(f"Failed: {filename} ({e})")

# Run the download function
download_dispatchprice()

✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202301.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202302.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202303.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202304.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202305.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202306.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202307.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202308.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202309.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202310.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202311.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202312.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202401.csv
✓ Already exists: data\raw\mms_dispatch_price\DISPATCHPRICE_202402.csv
✓ Alre

In [7]:
import pandas as pd
from glob import glob

# Path to all monthly price CSVs
price_files = sorted(glob(os.path.join(mms_dir, "DISPATCHPRICE_*.csv")))

print(f"Found {len(price_files)} monthly price files.")

# Load and concatenate
all_data = []

for file in price_files:
    try:
        df = pd.read_csv(file, skiprows=1)  # first row is metadata
        all_data.append(df)
    except Exception as e:
        print(f"Failed to read {file}: {e}")

df_price = pd.concat(all_data, ignore_index=True)

# Filter QLD1 only
df_price_qld = df_price[df_price["REGIONID"] == "QLD1"].copy()

# Save to cleaned folder
clean_price_path = os.path.join(cleaned_dir, "price_demand_qld_2023_2024.csv")
df_price_qld.to_csv(clean_price_path, index=False)

print(f"Saved cleaned QLD price data to: {clean_price_path}")
print(f"Shape: {df_price_qld.shape}")

Found 19 monthly price files.
Saved cleaned QLD price data to: data\cleaned\price_demand_qld_2023_2024.csv
Shape: (166494, 58)


##### 3. Renewable Generation Data (DISPATCH_UNIT_SCADA)

In [9]:
# Create SCADA folder
scada_dir = os.path.join(raw_dir, "mms_dispatch_scada")
os.makedirs(scada_dir, exist_ok=True)

def download_dispatch_scada(years=(2023, 2024), delay=1):
    base_url = "http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/{}/MMSDM_{}_{:02d}/MMSDM_Historical_Data_SQLLoader/DATA/{}.zip"

    for year in range(years[0], years[1] + 1):
        for month in range(1, 13):
            if year == 2024 and month > 12:
                continue
            month_str = f"{year}{month:02d}"
            filename = f"PUBLIC_DVD_DISPATCH_UNIT_SCADA_{month_str}010000"
            url = base_url.format(year, year, month, filename)

            zip_path = os.path.join(scada_dir, f"{filename}.zip")
            csv_path = os.path.join(scada_dir, f"DISPATCH_UNIT_SCADA_{month_str}.csv")

            if os.path.exists(csv_path):
                print(f"✓ Already exists: {csv_path}")
                continue

            try:
                print(f"⬇ Downloading: {filename}.zip")
                response = requests.get(url, headers=headers)
                response.raise_for_status()

                with open(zip_path, 'wb') as f:
                    f.write(response.content)

                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(scada_dir)

                extracted_csv = os.path.join(scada_dir, f"{filename}.CSV")
                os.rename(extracted_csv, csv_path)
                print(f"Extracted: {csv_path}")

                time.sleep(delay)

            except Exception as e:
                print(f"Failed: {filename} ({e})")

# Run the function
download_dispatch_scada()

✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202301.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202302.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202303.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202304.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202305.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202306.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202307.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202308.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202309.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202310.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202311.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202312.csv
✓ Already exists: data\raw\mms_dispatch_scada\DISPATCH_UNIT_SCADA_202401.csv

##### Download the Registration Excel File

In [11]:
# Folder to save the registration Excel file
reg_excel_path = os.path.join(reg_dir, "NEM_Registration_and_Exemption_List.xlsx")

registration_url = "https://www.aemo.com.au/-/media/files/electricity/nem/participant_information/nem-registration-and-exemption-list.xlsx"

try:
    print(f"⬇ Downloading generator registration list...")
    response = requests.get(registration_url, headers=headers)
    response.raise_for_status()
    with open(reg_excel_path, 'wb') as f:
        f.write(response.content)
    print(f"Saved to: {reg_excel_path}")
except Exception as e:
    print(f" Failed to download registration list: {e}")


⬇ Downloading generator registration list...
Saved to: data\registration\NEM_Registration_and_Exemption_List.xlsx


##### Load & Merge SCADA Files

In [13]:
# Get all SCADA CSV files
scada_files = sorted(glob(os.path.join(scada_dir, "DISPATCH_UNIT_SCADA_*.csv")))

print(f"Found {len(scada_files)} SCADA files")

# Load all into a single DataFrame
scada_data = []

for file in scada_files:
    try:
        df = pd.read_csv(file, skiprows=1, usecols=["SETTLEMENTDATE", "DUID", "SCADAVALUE"])
        scada_data.append(df)
    except Exception as e:
        print(f"Failed to read {file}: {e}")

df_scada = pd.concat(scada_data, ignore_index=True)
print(" SCADA data loaded:", df_scada.shape)

Found 19 SCADA files
 SCADA data loaded: (76427403, 3)


##### Load & Filter the Registration Excel File

In [15]:
xls = pd.ExcelFile(reg_excel_path)
print("Available sheet names:")
print(xls.sheet_names)

Available sheet names:
['Title Page', 'Registered Participants', 'Applications Received ', 'Ceasing Registration', 'Suspended Participants', 'PU and Scheduled Loads', 'Exemption - Intermediary', 'Exemption - small gen or IRS', 'Exemption - Central Dispatch', 'Ancillary Services', 'Wholesale Demand Response Units', 'Metering Coordinators', 'Dedicated Connection Asset']


In [16]:
# Load "Production Units (PU) and Scheduled Loads" tab
df_reg = pd.read_excel(reg_excel_path, sheet_name="PU and Scheduled Loads", skiprows=0)

# Keep only renewable units in QLD
renewables = ["Solar", "Wind", "Hydro"]
df_renewables_qld = df_reg[
    (df_reg["Region"] == "QLD1") &
    (df_reg["Fuel Source - Primary"].isin(renewables))
]

# Ensure only valid DUIDs (no NaN)
df_renewables_qld = df_renewables_qld[df_renewables_qld["DUID"].notna()]

print("Renewable QLD generators:", df_renewables_qld.shape)

Renewable QLD generators: (58, 20)


##### Merge SCADA with Registration & Filter

In [18]:
# Merge SCADA with registration by DUID
df_merged = pd.merge(df_scada, df_renewables_qld[["DUID", "Fuel Source - Primary", "Technology Type - Primary"]], on="DUID", how="inner")

print(" Merged renewable generation data:", df_merged.shape)

 Merged renewable generation data: (8107423, 5)


##### Save Final Dataset

In [20]:
# Convert settlement date to datetime
df_merged["SETTLEMENTDATE"] = pd.to_datetime(df_merged["SETTLEMENTDATE"])

# Save to cleaned folder
renewable_out_path = os.path.join(cleaned_dir, "renewable_generation_qld_2023_2024.csv")
df_merged.to_csv(renewable_out_path, index=False)

print(f" Saved renewable generation data to: {renewable_out_path}")

 Saved renewable generation data to: data\cleaned\renewable_generation_qld_2023_2024.csv


##### Prepare and Convert for Analysis

In [22]:
def prepare_registration_data():
    import pandas as pd
    import os
    import requests

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
    }

    # Paths
    reg_url = "https://www.aemo.com.au/-/media/files/electricity/nem/participant_information/nem-registration-and-exemption-list.xlsx"
    xlsx_path = os.path.join("data", "aemo_data", "NEM_Registration_and_Exemption_List.xlsx")
    csv_path = os.path.join("data", "analysis", "NEM_Registration.csv")

    os.makedirs("data/aemo_data", exist_ok=True)
    os.makedirs("data/analysis", exist_ok=True)

    # Download Excel
    try:
        print("⬇ Downloading latest registration file...")
        r = requests.get(reg_url, headers=headers)
        r.raise_for_status()
        with open(xlsx_path, "wb") as f:
            f.write(r.content)
        print(f" Excel saved: {xlsx_path}")
    except Exception as e:
        print(f" Download failed: {e}")
        return

    # Extract PU sheet and save as CSV
    try:
        df = pd.read_excel(xlsx_path, sheet_name="PU and Scheduled Loads")
        df.to_csv(csv_path, index=False)
        print(f" CSV prepared: {csv_path}")
    except Exception as e:
        print(f" Excel to CSV failed: {e}")

# Run it once
prepare_registration_data()

⬇ Downloading latest registration file...
 Excel saved: data\aemo_data\NEM_Registration_and_Exemption_List.xlsx
 CSV prepared: data\analysis\NEM_Registration.csv


##### Load and Preview the CSV

In [24]:
df_reg = pd.read_csv("data/analysis/NEM_Registration.csv")
df_reg.head()

,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Units,Aggregation,DUID,Reg Cap generation (MW),Max Cap generation (MW),Max ROC/Min generation,Reg Cap consumption (MW),Max Cap consumption (MW),Max ROC/Min consumption,Comments
0,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,1-3,Y,ADPBA1G,7.76,6.15,2,NaN,NaN,NaN,NaN
1,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Load,Market,Scheduled,Battery storage,Grid,Storage,Battery and Inverter,1-3,Y,ADPBA1L,7.76,6.15,2,NaN,NaN,NaN,NaN
2,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Hydro,Water,Renewable,Run of River,1-2,Y,ADPMH1,1.44,1,-,NaN,NaN,NaN,NaN
3,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Semi-Scheduled,Solar,Solar,Renewable,Photovoltaic Tracking Flat panel,"1-4, 5-9",Y,ADPPV1,24.75,19,4,NaN,NaN,NaN,NaN
4,South Australian Water Corporation,Adelaide Desalination Plant,SA1,Generating Unit,Market,Non-Scheduled,Solar,Solar,Renewable,Photovoltaic Flat panel,1-2,Y,ADPPV2,0.2,0.2,-,NaN,NaN,NaN,NaN


Data Availability Limitation: Why We Only Have Up to July 2024
Our original intent was to analyze the full 2-year window from January 2023 to December 2024. 
This could be due to:
1. At the time of data collection (May 2025), the latest available dispatch data was from July 2024
2. Files from August 2024 onward returned 404 Not Found, indicating they were not yet released

Hence, our cleaned datasets reflect the maximum available history, and still provide over 19 months of high-frequency data (5-minute intervals), which is more than sufficient for time-series modeling, exploratory analysis, and scenario simulations.